#0.셋업

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install konlpy
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git 
%cd Mecab-ko-for-Google-Colab/
!bash install_mecab-ko_on_colab190912.sh
%cd ../

     |████████████████████████████████| 19.4 MB 532 kB/s 
     |████████████████████████████████| 448 kB 21.3 MB/s 
Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 109, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 109 (delta 7), reused 10 (delta 3), pack-reused 91
Receiving objects: 100% (109/109), 1.27 MiB | 8.23 MiB/s, done.
Resolving deltas: 100% (46/46), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2022-04-21 00:08:36--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::3403:4be7, 2406:da00:ff00::22c0:3470, ...
Connecting to bitbucket.org (bitbucket.org)|104.192.141.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Loca

# 1.인트로

## 1.1 문제 유형 및 데이터 정리

## 1.2 패키지 로드

In [3]:
import numpy as np
import pandas as pd

from konlpy.tag import Mecab

from collections import Counter
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten

import matplotlib.pyplot as plt

## 1.3 데이터 로드

### 1.3.1 파일 로드

### 1.3.2 모델 로드

In [4]:
# 모델 불러오기

# path = '/content/drive/MyDrive/YGL/Final_Project/Final_Project_Sharing/민경태/Result/'
path = '/content/drive/MyDrive/YGL/Final_Project/Final_Project_Sharing/여행 코스 추천 시스템/모델 코드/모델 저장/'

model_sentence = tf.keras.models.load_model(path + 'model_catbot_sentence_e_5_dl_351331_1650423783.750069.h5')
model_human = tf.keras.models.load_model(path + 'model_catbot_human_e_5_dl_316464_1650414569.121128.h5')
model_thema = tf.keras.models.load_model(path + 'model_catbot_thema_e_5_dl_437818_1650421177.1240025.h5')
model_info = tf.keras.models.load_model(path + 'model_catbot_info_e_5_dl_219573_1650433522.0790615.h5')
# test_loss, test_acc = new_model.evaluate(x,  y, verbose=2)

### 1.3.3 단어장 로드

In [5]:
def vocab_load(path, file_name, tokenizer):
    # 단어장 불러오기
    with open(path + file_name) as json_file:
        word_index = json.load(json_file)
        tokenizer.word_index = word_index
    return tokenizer

In [6]:
import json

tokenizer_sent = vocab_load(path, 'sentence_wordIndex_v2.json', Tokenizer())
tokenizer_human = vocab_load(path, 'human_wordIndex_v2.json', Tokenizer())
tokenizer_thema = vocab_load(path, 'thema_wordIndex_v2.json', Tokenizer())
tokenizer_info = vocab_load(path, 'info_wordIndex_v1.json', Tokenizer())

# 2.챗봇 알고리즘

##2.1 제일 순위가 높은 인덱스 가져오기

In [14]:
mecab = Mecab()
stopwords = ['의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '도', '를', '으로', '자', '에', '와', '한', '하다']
max_len = 15

def sentence_sentiment_predict(new_sentence):
    new_sentence = mecab.morphs(new_sentence) # 형태소 분석 tokenizer

    new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거

    encoded = tokenizer_sent.texts_to_sequences([new_sentence]) # 텍스트 > 정수 시퀀스
    pad_new = pad_sequences(encoded, maxlen=max_len) # 패딩
    score = model_sentence.predict(pad_new)

    return np.argmax(score)

def human_sentiment_predict(new_sentence):
    new_sentence = mecab.morphs(new_sentence) # 형태소 분석 tokenizer

    new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거

    encoded = tokenizer_human.texts_to_sequences([new_sentence]) # 텍스트 > 정수 시퀀스
    pad_new = pad_sequences(encoded, maxlen=max_len) # 패딩
    score = model_human.predict(pad_new)

    return np.argmax(score)

def thema_sentiment_predict(new_sentence):
    new_sentence = mecab.morphs(new_sentence) # 형태소 분석 tokenizer

    new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거

    encoded = tokenizer_thema.texts_to_sequences([new_sentence]) # 텍스트 > 정수 시퀀스
    pad_new = pad_sequences(encoded, maxlen=max_len) # 패딩
    score = model_thema.predict(pad_new)

    return np.argmax(score)

def info_sentiment_predict(new_sentence):
    new_sentence = mecab.morphs(new_sentence) # 형태소 분석 tokenizer

    new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거

    encoded = tokenizer_info.texts_to_sequences([new_sentence]) # 텍스트 > 정수 시퀀스
    pad_new = pad_sequences(encoded, maxlen=max_len) # 패딩
    score = model_info.predict(pad_new)

    return np.argmax(score)

In [8]:
sent_list = ['추천', '대화', '정보']
human_list = ['None', '가족' , '어린이', '친구', '커플', '혼자']
thema_list = ['가을', '겨울', '공중', '도보', '봄', '사계절', '사진', '수상', '여름', '역사', '예술', '육상', '자연']
info_list = ['날씨', '시간', '위치']

In [9]:
text = '가족 여행 가기 좋은 곳'

# 추천 : 0, 대화 : 1, 정보 : 2
print(sent_list[int(sentence_sentiment_predict(text))])

#  None : 0, 가족 : 1, 어린이 : 2, 친구 : 3, 커플 : 4, 혼자 : 5
print(human_list[int(human_sentiment_predict(text))])

# 가을 : 0, 겨울 : 1, 공중 : 2, 도보 : 3, 봄 : 4, 사계절 : 5, 사진 : 6, 수상 : 7, 여름 : 8, 역사 : 9, 예술 : 10, 육상 : 11, 자연 : 12
print(thema_list[int(thema_sentiment_predict(text))])


추천
None
도보


##2.2 순위 출력

- threshold를 정해 일정 값 보다 낮으면 포함시키지 않음 (>0.9)

In [10]:
mecab = Mecab()
stopwords = ['의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '도', '를', '으로', '자', '에', '와', '한', '하다']
max_len = 15

def sentence_sentiment_predict(new_sentence):
    new_sentence = mecab.morphs(new_sentence) # 형태소 분석 tokenizer

    new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거

    encoded = tokenizer_sent.texts_to_sequences([new_sentence]) # 텍스트 > 정수 시퀀스
    pad_new = pad_sequences(encoded, maxlen=max_len) # 패딩
    score = model_sentence.predict(pad_new)

    list_score = list(range(len(score[0])))

    df_rank = pd.DataFrame()

    df_rank['idx'] = list_score
    df_rank['value'] = score[0].tolist()
    df_rank['rank'] = df_rank['value'].rank(ascending=False)

    top5 = df_rank.sort_values(by='rank').iloc[:5, :]

    return top5

def human_sentiment_predict(new_sentence):
    new_sentence = mecab.morphs(new_sentence) # 형태소 분석 tokenizer

    new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거

    encoded = tokenizer_human.texts_to_sequences([new_sentence]) # 텍스트 > 정수 시퀀스
    pad_new = pad_sequences(encoded, maxlen=max_len) # 패딩
    score = model_human.predict(pad_new)

    list_score = list(range(len(score[0])))

    df_rank = pd.DataFrame()

    df_rank['idx'] = list_score
    df_rank['value'] = score[0].tolist()
    df_rank['rank'] = df_rank['value'].rank(ascending=False)

    top5 = df_rank.sort_values(by='rank').iloc[:5, :]

    return top5

def thema_sentiment_predict(new_sentence):
    new_sentence = mecab.morphs(new_sentence) # 형태소 분석 tokenizer

    new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거

    encoded = tokenizer_thema.texts_to_sequences([new_sentence]) # 텍스트 > 정수 시퀀스
    pad_new = pad_sequences(encoded, maxlen=max_len) # 패딩
    score = model_thema.predict(pad_new)
    
    list_score = list(range(len(score[0])))

    df_rank = pd.DataFrame()

    df_rank['idx'] = list_score
    df_rank['value'] = score[0].tolist()
    df_rank['rank'] = df_rank['value'].rank(ascending=False)

    top5 = df_rank.sort_values(by='rank').iloc[:5, :]

    return top5

In [11]:
text = '사진'

# 추천 : 0, 대화 : 1, 정보 : 2
# print(sent_list[int(sentence_sentiment_predict(text))])
display(sentence_sentiment_predict(text))

# 
# print(human_list[int(human_sentiment_predict(text))])
display(human_sentiment_predict(text))

# 가을 : 0, 겨울 : 1, 공중 : 2, 도보 : 3, 봄 : 4, 사계절 : 5, 사진 : 6, 수상 : 7, 여름 : 8, 역사 : 9, 예술 : 10, 육상 : 11, 자연 : 12
# print(thema_list[int(thema_sentiment_predict(text))])
display(thema_sentiment_predict(text))

,idx,value,rank
1,1,0.999471,1.0
2,2,0.000309,2.0
0,0,0.000220,3.0


,idx,value,rank
0,0,0.998322,1.0
2,2,0.001120,2.0
5,5,0.000276,3.0
3,3,0.000146,4.0
1,1,0.000135,5.0


,idx,value,rank
6,6,0.926129,1.0
10,10,0.038297,2.0
8,8,0.015945,3.0
2,2,0.009026,4.0
4,4,0.004093,5.0


##2.3 여행 챗봇 알고리즘

In [12]:
key_dict = {
    '구성원' : [],
    '태그'   : []
}

In [15]:
input_text = '제주도 가게 언제 열어?'

# 추천 : 0, 대화 : 1, 정보 : 2
# None : 0, 가족 : 1, 어린이 : 2, 친구 : 3, 커플 : 4, 혼자 : 5
# 가을 : 0, 겨울 : 1, 공중 : 2, 도보 : 3, 봄 : 4, 사계절 : 5, 사진 : 6, 수상 : 7, 여름 : 8, 역사 : 9, 예술 : 10, 육상 : 11, 자연 : 12
# 날씨 : 0, 시간 : 1, 위치 : 2

print(sent_list[int(sentence_sentiment_predict(input_text))])

# 추천
if int(sentence_sentiment_predict(input_text)) == 0: 
    print('추천')

    # 문장의 구성원 받기
    key_dict['구성원'].append(int(human_sentiment_predict(input_text)))

    # 반복된 구성원 키워드에 대한 중복 제거
    key_dict['구성원'] = list(set(key_dict['구성원']))

    # 문장의 태그 받기
    key_dict['태그'].append(int(thema_sentiment_predict(input_text)))

    # 반복된 태그 키워드에 대한 중복 제거
    key_dict['태그'] = list(set(key_dict['태그']))

    # 만약 구성원에서 None을 받은 경우
    if key_dict['구성원'][0] == 0: 
        print('누구와 가고 싶은 여행인지 자세히 알려주시면 알맞는 추천을 해드릴게요.')

    # 딕셔너리에 값이 없는 경우
    if len(key_dict['구성원']) == 0 or len(key_dict['태그']) == 0:
        print('추천을 해드리기 어려울 것 같아요. 어떤 여행을 하고 싶으신지 알려주세요.')

elif int(sentence_sentiment_predict(input_text)) == 1:
    print('대화')
    print('안녕하세요. 반가워요! 저는 여러분들을 위한 여행용 챗봇 입니다.')

else:
    print('정보')
    print(info_list[int(info_sentiment_predict(input_text))])


print('결과---------------------------')
print(key_dict)


정보
정보
시간
결과---------------------------
{'구성원': [], '태그': []}


In [17]:
def make_input_dataframe(key_dict):
    col = ['역사', '자연', '사진', '도보', '예술', '육상', '수상', '공중', '봄', '여름', '가을', '겨울',
       '사계절', '어린이', '청년', '중년', '노년', '혼자', '친구', '커플', '부모', '가족']

    df_input_data = pd.DataFrame([0] * len(col), index=col).T

    if key_dict['구성원'][0] != 0:
        for i in key_dict['구성원']:
            # print(human_list[i])
            df_input_data[human_list[i]] += 1

    for i in key_dict['태그']:
        # print(thema_list[i])
        df_input_data[thema_list[i]] += 1

    display(df_input_data)
    return df_input_data

In [21]:
print('안녕하세요 여행용 챗봇 입니다. 원하시는 여행을 알려주세요.')

key_dict = {
    '구성원' : [],
    '태그'   : []
}


while 1:

    # input_text = input()
    input_text = '가족들이랑 자전거 여행하고 싶어'

    if input_text == '실행':

        # 딕셔너리에 값이 없는 경우
        if len(key_dict['구성원']) == 0 or len(key_dict['태그']) == 0:
            print('\n추천을 해드리기 어려울 것 같아요. 어떤 여행을 하고 싶으신지 알려주세요.')

        if len(key_dict['구성원']) > 0 and len(key_dict['태그']) > 0:
            print('\n추천을 시작합니다..')
            df_input_data = make_input_dataframe(key_dict)

            save_path = '/content/drive/MyDrive/YGL/Final_Project/Final_Project_Sharing/손찬우/'

            df_input_data.to_csv(save_path + 'df_input_data.csv')
            break

    elif input_text == '종료':    
        print('\n프로그램을 종료합니다. 감사합니다.')   
        break

    else:
        if int(sentence_sentiment_predict(input_text)) == 0: 
            print('추천\n')

            # 문장의 구성원 받기
            key_dict['구성원'].append(int(human_sentiment_predict(input_text)))

            # 반복된 구성원 키워드에 대한 중복 제거
            key_dict['구성원'] = list(set(key_dict['구성원']))

            # 문장의 태그 받기
            key_dict['태그'].append(int(thema_sentiment_predict(input_text)))

            # 반복된 태그 키워드에 대한 중복 제거
            key_dict['태그'] = list(set(key_dict['태그']))

            # 만약 구성원에서 None을 받은 경우
            if key_dict['구성원'][0] == 0: 
                print('\n누구와 가고 싶은 여행인지 자세히 알려주시면 알맞는 추천을 해드릴게요.')
                # key_dict['구성원'] = []

            if len(key_dict['구성원']) > 0 and len(key_dict['태그']) > 0:
                print('\n여행 추천 준비가 끝났습니다. 입력으로 <실행>을 해주세요')

            print(key_dict)

        elif int(sentence_sentiment_predict(input_text)) == 1:
            print('대화')
            print('\n안녕하세요. 반가워요! 저는 여러분들을 위한 여행용 챗봇 입니다.')

        else:
            print('정보')
            print(info_list[int(info_sentiment_predict(input_text))])

        
    


안녕하세요 여행용 챗봇 입니다. 원하시는 여행을 알려주세요.
자전거 여행 가고 싶어
추천


누구와 가고 싶은 여행인지 자세히 알려주시면 알맞는 추천을 해드릴게요.

여행 추천 준비가 끝났습니다. 입력으로 <실행>을 해주세요
{'구성원': [0], '태그': [11]}
실행

추천을 시작합니다..


,역사,자연,사진,도보,예술,육상,수상,공중,봄,여름,...,사계절,어린이,청년,중년,노년,혼자,친구,커플,부모,가족
0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# 3.학습 & 예측

# 4.성능 지표 확인

# 5.저장

## 5.1 github에 저장

## 5.2 파일로 저장